In [1]:
import os
import time
import obspy
import pandas as pd

from obspy.core.inventory import Inventory, Network, Station, Channel, Site

In [2]:
class NetworkInfo:
    def __init__(self, code, description="Default description"):
        self.code = code
        self.description = description

class StationInfo:
    def __init__(self, code, latitude, longitude, elevation, depth, creation_date=obspy.UTCDateTime(time.time())):
        self.code = code
        self.latitude = latitude
        self.longitude = longitude
        self.elevation = elevation
        self.depth = depth
        self.creation_date = creation_date

class ChannelInfo:
    def __init__(self, code, azimuth, dip, sample_rate, location_code='00'):
        self.code = code
        self.azimuth = azimuth
        self.dip = dip
        self.sample_rate = sample_rate
        self.location_code = location_code

def generate_inventory(source, network_info, stations_info, channels_info, path):
    inv = Inventory(
        # We'll add networks later.
        networks=[],
        # The source should be the id whoever create the file.
        source=source)

    net = Network(
        # This is the network code according to the SEED standard.
        code=network_info.code,
        # A list of stations. We'll add one later.
        stations=[],
        description=network_info.description)

    for sta_info in stations_info:
        sta = Station(
                # This is the station code according to the SEED standard.
                code=sta_info.code,
                latitude=sta_info.latitude,
                longitude=sta_info.longitude,
                elevation=sta_info.elevation,
                creation_date=sta_info.creation_date,
                site=Site(name=sta_info.code))

        for cha_info in channels_info:
            cha = Channel(
                # This is the channel code according to the SEED standard.
                code=cha_info.code,
                # This is the location code according to the SEED standard.
                location_code=cha_info.location_code,
                # Note that these coordinates can differ from the station coordinates.
                latitude=sta_info.latitude,
                longitude=sta_info.longitude,
                elevation=sta_info.elevation,
                depth=sta_info.depth,
                azimuth=cha_info.azimuth,
                dip=cha_info.dip,
                sample_rate=cha_info.sample_rate)

            sta.channels.append(cha)

        net.stations.append(sta)

    inv.networks.append(net)

    inv.write(os.path.join(path, '{}.xml'.format(source)), format="stationxml", validate=True)

In [12]:
def lena_generate():
    df = pd.read_excel('Station_data.xlsx')
    
    codes = ['HHE', 'HHN', 'HHZ']
    azs = [90, 0, 0]
    dips = [0, 0, -90]
    sampling_rate = 125
    location = '00'
    
    network = NetworkInfo('LD', 'Modern Lena network')
    
    stations = [StationInfo(code, latit, longit, elev, 0) \
                for code, latit, longit, elev in zip(df['Name'], df['Latitude'], df['Longitude'], df['Altitude'])]
    
    channels = [ChannelInfo(code, az, dip, sampling_rate, location) \
                for code, az, dip, in zip(codes, azs, dips)]
                
    path_out = 'station_xml_folder'
    generate_inventory('LD_HH_stations', network, stations, channels, path_out)
    
    
new_lena_generate()